In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
import biovis.plotbio as plb
import biovis.netgraph as ng
import biovis.cmaps as cm

%load_ext autoreload 

# check on every execution call
%autoreload 2 



In [3]:
# load network structure

#base_dir = "/data/mat/slg/ice"
#cells_file_name = base_dir+"/sims/layer4/ll2/net/cells.csv"
#cell_models_file_name = base_dir+"/sims/layer4/ll2/net/cell_models.csv"
#morph_dir = base_dir+"/cell_models/biophysical/morph_segs"

cells_file_name ="./column_net/structure/cells.csv"
cell_models_file_name = "./column_net/structure/cell_models.csv"
morph_dir = "./column_net/structure/morph_segs"

nodes_df, cm_df = ng.load_nodes(cells_file_name, cell_models_file_name)
cm_bio_df = cm_df[cm_df['level_of_detail']=='biophysical']
morphologies, soma_sizes = ng.load_morphologies(morph_dir, cm_bio_df)

# added soma_sizes for intfire neurons:
soma_sizes[100000101] = 10.0
soma_sizes[100000102] = 10.0


...importing nodes:  50770
# unique morphologies:  0


In [6]:
#Make selections:

mask_biophysical = "level_of_detail=='biophysical'"
mask_intfire = "level_of_detail=='intfire'"

bio_cells_df = nodes_df.query(mask_biophysical)
iaf_cells_df = nodes_df.query(mask_intfire)

print "number of cells satisfying selection:", len(bio_cells_df.index)
print "number of cells satisfying selection:", len(iaf_cells_df.index)




number of cells satisfying selection: 0
number of cells satisfying selection: 0


In [7]:
#Initialize figure object
fig = plb.figure()

fig.set_bgcolor('black')    #Set figure background



... initializing canvas ...
...setting gbcolor:  black


In [8]:
#Create layer slices
layer_depths = [100,310,420,650,850]  # values will display as negative, i.e. from top of cortex
layer_colors = ['red','blue','green','magenta','cyan'] 
layer_alpha = 60
fig.set_layers(layer_depths, layer_colors, layer_alpha)



...setting layers...


In [9]:
#Create frame on outside of column; standard size is 1mm^3 and should not be changed
frame=[[500,0,500], [500,0,-500], [-500,0,-500], [-500,0,500], [500,1000,500], [500,1000,-500], [-500,1000,-500], [-500,1000,500]]
frame_colour = [(255,255,255)]
fig.set_frame(frame, frame_colour) # xyz volume block to place in the window. To remove use: fig.set_frame([],[]) 



...setting frame...


In [ ]:
fig.show()

In [ ]:
# choose colors:
# http://www.discoveryplayground.com/computer-programming-for-kids/rgb-colors/
# http://matplotlib.org/examples/color/named_colors.html
# cm.colors_lib # dictionary of all named colors as color_name:(r,g,b)
# put desired colors in the list below to use in a colormap:
# may use multiple lists
color_list = ['lightseagreen', 'brown', 'cadetblue', 'coral', 'blueviolet', 'beige', 'green','sandybrown','sienna'] 

unique_attrs = bio_cells_df['pop_name'].unique() # find unique labels from a particular column
cmap_pop = cm.make_color_map(unique_attrs,color_list) # make a color map for these labels
cmap_ei = {'e':'red','i':'blue'} # example of a manually created colormap
cmap_pop['LIF_exc']='red' # add to a colormap
cmap_pop['LIF_inh']='blue'

cmap_pop

In [ ]:
bio_inh_cells_df = nodes_df.query("level_of_detail=='biophysical' & ei=='i'")
print "number of biophysical inhibitory cells:", len(bio_inh_cells_df.index)

In [64]:
#plot selected cells

#fig.plot_morph(bio_cells_df[::400], morphologies, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies
#fig.plot_somas(bio_cells_df[::400], soma_sizes, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies
#fig.plot_somas(iaf_cells_df[::10], soma_sizes, cmap_ei,'ei')    #Pass query into plotting routine along with raw morphologies
#fig.plot_somas(bio_cells_df, soma_sizes, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies

fig.plot_morph3D(bio_inh_cells_df[::50], morphologies, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies
#fig.plot_somas(bio_inh_cells_df[::10], soma_sizes, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies
fig.plot_morph(bio_inh_cells_df[::50], morphologies, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies






...drawing 3D segments...
... processing cell:  ready to display 3d segments!


In [65]:
fig.show()


... showing ...


In [61]:
fig.clear()

In [107]:
from isee_engine.tools import tio

# examine intracortical connections:

con_ic_file_name = base_dir+"/sims/layer4/ll2/net/connections_ic.h5"

connections = tio.load_ij_connections(con_ic_file_name) # just a file handle
indptr = connections['indptr'][...]  # read from file all indptr - has info on which lines to read from hdf5

tar_gid = 78 
src_gids = connections['src_gids'][indptr[tar_gid]:indptr[tar_gid+1]]


In [108]:
tar_cell_df = nodes_df.loc[[tar_gid]]
src_cells_df = nodes_df.loc[src_gids]


In [109]:
fig.plot_morph(tar_cell_df, morphologies, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies
fig.plot_somas(tar_cell_df, soma_sizes, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies
fig.plot_somas(src_cells_df, soma_sizes, cmap_pop,'pop_name')    #Pass query into plotting routine along with raw morphologies


... processing cell:  ready to display 3d segments!
... processing cell:  ready to display spherical somata!
... processing cell:  100 200 300 400 ready to display spherical somata!


In [111]:
fig.show()

... showing ...


In [104]:
fig.clear()

In [5]:
# examine thalamocortical connections:
from isee_engine.tools import tio
from scipy.sparse import csr_matrix

con_tc_file_name = base_dir+"/sims/layer4/ll2/net/connections_tc.h5"
con_tc = tio.load_ij_connections(con_tc_file_name) # just a file handle

tc_csr  = csr_matrix((  con_tc['nsyns'], con_tc['src_gids'], con_tc['indptr']))

tc_csc = tc_csr.tocsc() # connections organized 


In [20]:
lgn_gid = 1446 # chose lgn cell id

indptr_csc = tc_csc.indptr
targets = tc_csc.indices
v1_gids = targets[indptr_csc[lgn_gid]:indptr_csc[lgn_gid+1]]
v1_df = nodes_df.loc[v1_gids]


In [21]:
fig.plot_somas(v1_df, soma_sizes, cmap_pop,'pop_name')    

... processing cell:  100 200 300 400 ready to display spherical somata!


In [ ]:
fig.show()

In [127]:
fig.clear()

In [ ]:
# plot connection matrix:
import matplotlib.pyplot as plt
fig2 = plt.figure(2)
plt.spy(tc_csc,markersize=2)
plt.xlabel('src_gid')
plt.ylabel('tar_gid')
plt.show()
